In [ ]:
import os
# set the current working directory to the deployed package folder. This is required by isaac.
# This cell should only run once.
os.chdir("../../../")
os.getcwd()

In [ ]:
from IPython.display import display
import json
import numpy as np

from engine.pyalice import Application, Codelet
from engine.pyalice.gui.composite_widget import CompositeWidget

np.set_printoptions(precision=3)

In [ ]:
# A Python codelet for joint control through widget
class JointPositionControl(Codelet):
    def start(self):
        self.rx = self.isaac_proto_rx("CompositeProto", "state")
        self.tx = self.isaac_proto_tx("CompositeProto", "command")

        joints = self.config.joints
        limits = self.config.limits
        measure = self.config.measure
        self._widget = CompositeWidget(joints, measure, limits)
        if self._widget is None:
            report_failure("Cannot create valid widget")
            return
        display(self._widget.panel)

        self.tick_periodically(0.1)

    def tick(self):
        state_msg = self.rx.message
        if state_msg is None:
            return
        self._widget.composite = state_msg

        self.tx._msg = self._widget.composite
        if self.tx._msg is not None:
            self.tx.publish()
            
# set kinematic file and get list of joints
kinematic_file = "apps/assets/kinematic_trees/ur10.kinematic.json"
joints = []
with open(kinematic_file,'r') as fd:
    kt = json.load(fd)
    for link in kt['links']:
        if 'motor' in link and link['motor']['type'] != 'constant':
            joints.append(link['name'])
print(joints)

# Obtain the UR10 IP from UR Console and update this
UR10_IP = "10.32.221.190"

UR10 Hardware Joint Control
======

In [ ]:
app = Application(name="simple_joint_control_ur10_hardware")

# load lqr subgraphcs
app.load(filename="packages/planner/apps/multi_joint_lqr_control.subgraph.json", prefix="lqr")
lqr_interface = app.nodes["lqr.subgraph"]["interface"]

# configs
app.nodes["lqr.kinematic_tree"]["KinematicTree"].config.kinematic_file = kinematic_file
lqr_planner = app.nodes["lqr.local_plan"]["MultiJointLqrPlanner"]
lqr_planner.config.speed_min = [-0.5] * len(joints)
lqr_planner.config.speed_max = [0.5] * len(joints)
lqr_planner.config.acceleration_min = [-0.5] * len(joints)
lqr_planner.config.acceleration_max = [0.5] * len(joints)

# add pycodelet for joint position control
widget_node = app.add("command_generator")
joint_commander = widget_node.add(JointPositionControl)
joint_commander.config.joints = joints
joint_commander.config.limits = [[-2*np.pi, 2*np.pi]] * len(joints)
joint_commander.config.measure = 'position'
app.connect(joint_commander, "command", lqr_interface, "joint_target")

# add ur10 driver codelet
app.load_module("universal_robots")
driver = app.add("ur").add(app.registry.isaac.universal_robots.UniversalRobots)
# configs
driver.config.kinematic_tree = "lqr.kinematic_tree"
driver.config.control_mode = "joint position"
driver.config.robot_ip = UR10_IP
driver.config.tick_period = "125Hz"
# edges
app.connect(driver, "arm_state", lqr_interface, "joint_state")
app.connect(driver, "arm_state", joint_commander, "state")
app.connect(lqr_interface, "joint_command", driver, "arm_command")

app.start()

In [ ]:
app.stop()

UR10 Digital IO Control
=======

In [ ]:
app = Application(name="io_control_ur10_hardware")

# load lqr subgraphcs
app.load(filename="packages/planner/apps/multi_joint_lqr_control.subgraph.json", prefix="lqr")
lqr_interface = app.nodes["lqr.subgraph"]["interface"]

# configs
app.nodes["lqr.kinematic_tree"]["KinematicTree"].config.kinematic_file = kinematic_file
lqr_planner = app.nodes["lqr.local_plan"]["MultiJointLqrPlanner"]
lqr_planner.config.speed_min = [-0.5] * len(joints)
lqr_planner.config.speed_max = [0.5] * len(joints)
lqr_planner.config.acceleration_min = [-0.5] * len(joints)
lqr_planner.config.acceleration_max = [0.5] * len(joints)

# add ur10 driver codelet
app.load_module("universal_robots")
driver = app.add("ur").add(app.registry.isaac.universal_robots.UniversalRobots)
# configs
driver.config.kinematic_tree = "lqr.kinematic_tree"
driver.config.robot_ip = UR10_IP
driver.config.tick_period = "100Hz"

# add pycodelet for digitalIO control
io_names = driver.config.tool_digital_out_names

widget_node = app.add("command_generator")
io_commander = widget_node.add(JointPositionControl)
io_commander.config.joints = io_names
io_commander.config.limits = [[0, 1]] * len(io_names)
io_commander.config.measure = 'none'
app.connect(io_commander, "command", driver, "io_command")
app.connect(driver, "io_state", io_commander, "state")

app.start()

In [ ]:
app.stop()